### Configuration et préparation des données

In [ ]:
import dspy
import warnings
warnings.filterwarnings('ignore')

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

In [ ]:
# Données d'entraînement
trainset = [
    {"ticket": "Mon ordinateur ne démarre plus depuis ce matin. J'ai une présentation importante dans 2 heures.", "category": "Hardware", "priority": "Urgent"},
    {"ticket": "Je n'arrive pas à me connecter à l'imprimante du 3e étage. Ça peut attendre.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le VPN ne fonctionne plus. Impossible d'accéder aux fichiers du serveur.", "category": "Network", "priority": "High"},
    {"ticket": "J'ai oublié mon mot de passe Outlook. Je peux utiliser le webmail.", "category": "Account", "priority": "Medium"},
    {"ticket": "Le site web affiche une erreur 500. Les clients ne peuvent plus commander!", "category": "Application", "priority": "Critical"},
    {"ticket": "Ma souris sans fil ne répond plus bien. Les piles sont faibles.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le système de paie ne calcule pas les heures supplémentaires. C'est la fin du mois.", "category": "Application", "priority": "Urgent"},
    {"ticket": "J'aimerais une mise à jour de mon logiciel Adobe quand vous aurez le temps.", "category": "Software", "priority": "Low"},
    {"ticket": "Le serveur de base de données est très lent. Toute la production est impactée.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "Je ne reçois plus les emails. J'attends des réponses de fournisseurs.", "category": "Email", "priority": "High"},
    {"ticket": "Mon écran externe ne s'affiche plus. Je peux travailler sur le laptop.", "category": "Hardware", "priority": "Medium"},
    {"ticket": "Le wifi de la salle A ne fonctionne pas. Réunion avec des externes dans 30 min.", "category": "Network", "priority": "Urgent"},
    {"ticket": "Je voudrais installer Slack pour mieux collaborer avec l'équipe.", "category": "Software", "priority": "Medium"},
    {"ticket": "Le système de sauvegarde a échoué cette nuit selon le rapport.", "category": "Infrastructure", "priority": "High"},
    {"ticket": "Mon clavier a une touche qui colle. C'est gérable mais ennuyeux.", "category": "Peripherals", "priority": "Low"}
]

# Données de validation
valset = [
    {"ticket": "Le serveur de fichiers est inaccessible. Personne ne peut travailler.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.", "category": "Account", "priority": "Urgent"},
    {"ticket": "L'écran de mon collègue en vacances clignote. On peut attendre.", "category": "Hardware", "priority": "Low"},
    {"ticket": "Le CRM plante quand j'essaie d'exporter les contacts.", "category": "Application", "priority": "High"},
    {"ticket": "Je voudrais changer ma photo de profil quand vous aurez un moment.", "category": "Account", "priority": "Low"},
    {"ticket": "La vidéoconférence ne fonctionne pas. Réunion avec New York dans 10 minutes!", "category": "Application", "priority": "Critical"},
    {"ticket": "Mon antivirus affiche un message d'expiration mais tout fonctionne.", "category": "Software", "priority": "Medium"}
]

# Catégories et priorités possibles
CATEGORIES = ["Hardware", "Software", "Network", "Application", "Infrastructure", "Account", "Email", "Peripherals"]
PRIORITIES = ["Low", "Medium", "High", "Urgent", "Critical"]

print(f"📊 Données chargées : {len(trainset)} entraînement, {len(valset)} validation")
print(f"📦 {len(CATEGORIES)} catégories, {len(PRIORITIES)} priorités")

# Partie 7: Patterns avancés (optionnel)

## 7.1 Introduction aux patterns avancés

Cette section couvre des **patterns de production** pour rendre vos modules DSPy plus robustes, fiables et performants.

### Pourquoi utiliser ces patterns?

En production, les LLMs peuvent :
- ❌ Générer des sorties invalides (mauvais format, valeurs hors limites)
- ❌ Échouer temporairement (timeout, rate limiting)
- ❌ Produire des résultats incohérents
- ❌ Être indisponibles (downtime API)

Les **patterns avancés** permettent de gérer ces situations.

### Patterns couverts

1. **Validation** : Vérifier et corriger les sorties invalides
2. **Retry** : Réessayer automatiquement en cas d'erreur
3. **Fallback** : Utiliser un modèle de secours si le principal échoue
4. **Ensemble** : Combiner plusieurs prédictions pour plus de robustesse
5. **Chaînage avec contrôle** : Pipelines complexes avec branchement conditionnel

Ces patterns sont **optionnels** mais fortement recommandés pour les applications de production.

## 6.2 Pattern de validation

Le **pattern de validation** vérifie que les sorties du LLM respectent les contraintes de votre application.

### Problème

Les LLMs peuvent générer :
- Des catégories qui n'existent pas ("Matériel" au lieu de "Hardware")
- Des priorités invalides ("Très urgent" au lieu de "Urgent")
- Des formats incorrects (minuscules au lieu de majuscules)

### Solution

Créer un module qui :
1. Exécute la prédiction
2. **Valide** la sortie
3. **Corrige** ou **rejette** si invalide
4. Optionnellement, **réessaye** avec des instructions clarifiées

### Exemple: classifier avec validation

In [ ]:
class ValidatedTicketClassifier(dspy.Module):
    """
    Classifier avec validation des sorties
    """
    
    def __init__(self):
        super().__init__()
        self.classifier = dspy.ChainOfThought(TicketClassifier)
        
        # Valeurs valides
        self.valid_categories = set(cat.lower() for cat in CATEGORIES)
        self.valid_priorities = set(pri.lower() for pri in PRIORITIES)
    
    def validate_and_correct(self, category, priority):
        """
        Valide et corrige les sorties si nécessaire
        
        Returns:
            (category, priority, is_valid)
        """
        category_lower = category.strip().lower()
        priority_lower = priority.strip().lower()
        
        is_valid = True
        
        # Validation de la catégorie
        if category_lower not in self.valid_categories:
            # Essayer de trouver une correspondance approximative
            if 'hard' in category_lower or 'matér' in category_lower:
                category = 'Hardware'
            elif 'soft' in category_lower or 'logic' in category_lower:
                category = 'Software'
            elif 'réseau' in category_lower or 'network' in category_lower:
                category = 'Network'
            elif 'compte' in category_lower or 'account' in category_lower:
                category = 'Account'
            else:
                # Par défaut, mettre "Other"
                category = 'Other'
                is_valid = False
        else:
            # Normaliser la casse
            category = next(c for c in CATEGORIES if c.lower() == category_lower)
        
        # Validation de la priorité
        if priority_lower not in self.valid_priorities:
            # Essayer de mapper
            if 'critic' in priority_lower or 'critique' in priority_lower:
                priority = 'Critical'
            elif 'urgent' in priority_lower:
                priority = 'Urgent'
            elif 'high' in priority_lower or 'haut' in priority_lower:
                priority = 'High'
            elif 'medium' in priority_lower or 'moyen' in priority_lower:
                priority = 'Medium'
            else:
                priority = 'Low'
                is_valid = False
        else:
            # Normaliser la casse
            priority = next(p for p in PRIORITIES if p.lower() == priority_lower)
        
        return category, priority, is_valid
    
    def forward(self, ticket):
        # Prédiction initiale
        result = self.classifier(ticket=ticket)
        
        # Validation et correction
        category, priority, is_valid = self.validate_and_correct(
            result.category,
            result.priority
        )
        
        return dspy.Prediction(
            category=category,
            priority=priority,
            is_valid=is_valid
        )

print("✅ Classe ValidatedTicketClassifier définie")

### Test du classifier avec validation

In [ ]:
print("✅ Test du classifier avec validation\n")

# Configurer le modèle
dspy.configure(lm=lm_ollama_llama)

# Créer le classifier validé
validated_classifier = ValidatedTicketClassifier()

# Tester sur quelques exemples
test_examples = [
    "Mon imprimante ne fonctionne pas",
    "Besoin d'accès au serveur de fichiers",
    "Le réseau est très lent, urgent!"
]

for i, ticket in enumerate(test_examples, 1):
    result = validated_classifier(ticket=ticket)
    status = "✅ Valide" if result.is_valid else "⚠️ Corrigé"
    print(f"{i}. {ticket}")
    print(f"   → {result.category} | {result.priority} | {status}\n")

# Évaluer sur l'ensemble de validation
print("📊 Évaluation sur l'ensemble de validation:")
score = evaluate_module(validated_classifier, val_examples, exact_match_metric)
print(f"   Score: {score:.2%}")

# Compter les corrections effectuées
corrections = 0
for example in val_examples[:10]:
    result = validated_classifier(ticket=example['ticket'])
    if not result.is_valid:
        corrections += 1

print(f"   Corrections appliquées: {corrections}/10 sur les premiers exemples")

## 6.3 Pattern de retry (réessayer en cas d'erreur)

Le **pattern de retry** réessaye automatiquement une opération en cas d'échec temporaire.

### Problème

Les APIs LLM peuvent échouer pour diverses raisons :
- Timeout réseau
- Rate limiting (trop de requêtes)
- Surcharge temporaire du service
- Erreurs intermittentes

### Solution

Implémenter une logique de retry avec :
1. **Nombre maximum de tentatives** (ex: 3)
2. **Délai exponentiel** entre les tentatives (1s, 2s, 4s)
3. **Gestion des erreurs** spécifiques

### Exemple: classifier avec retry

In [ ]:
import time

class RetryTicketClassifier(dspy.Module):
    """
    Classifier avec logique de retry en cas d'erreur
    """
    
    def __init__(self, max_retries=3, initial_delay=1.0):
        super().__init__()
        self.classifier = dspy.ChainOfThought(TicketClassifier)
        self.max_retries = max_retries
        self.initial_delay = initial_delay
    
    def forward(self, ticket):
        last_error = None
        
        for attempt in range(self.max_retries):
            try:
                # Tentative de prédiction
                result = self.classifier(ticket=ticket)
                
                # Succès - retourner le résultat
                return dspy.Prediction(
                    category=result.category,
                    priority=result.priority,
                    attempts=attempt + 1
                )
                
            except Exception as e:
                last_error = e
                
                # Si c'est la dernière tentative, on va lever l'erreur
                if attempt == self.max_retries - 1:
                    break
                
                # Calculer le délai avec backoff exponentiel
                delay = self.initial_delay * (2 ** attempt)
                
                print(f"⚠️ Tentative {attempt + 1} échouée: {e}")
                print(f"   Nouvelle tentative dans {delay:.1f}s...")
                
                time.sleep(delay)
        
        # Toutes les tentatives ont échoué
        raise Exception(f"Échec après {self.max_retries} tentatives: {last_error}")

print("✅ Classe RetryTicketClassifier définie")

## 6.4 Pattern de fallback (modèle de secours)

Le **pattern de fallback** utilise un modèle de secours si le modèle principal échoue.

### Problème

- Le modèle principal (API payante) peut être indisponible
- Vous voulez garantir la disponibilité du service
- Budget limité : utiliser un modèle local en secours

### Solution

Créer un module qui :
1. Essaye avec le **modèle principal** (haute qualité)
2. Si échec, bascule vers le **modèle de secours** (local/gratuit)
3. Retourne quelle stratégie a été utilisée

### Exemple: classifier avec fallback

In [ ]:
class FallbackTicketClassifier(dspy.Module):
    """
    Classifier avec fallback vers un modèle de secours
    """
    
    def __init__(self, primary_lm, fallback_lm):
        super().__init__()
        self.primary_lm = primary_lm
        self.fallback_lm = fallback_lm
        self.signature = TicketClassifier
    
    def forward(self, ticket):
        # Tentative avec le modèle principal
        try:
            with dspy.settings.context(lm=self.primary_lm):
                predictor = dspy.ChainOfThought(self.signature)
                result = predictor(ticket=ticket)
                
                return dspy.Prediction(
                    category=result.category,
                    priority=result.priority,
                    model_used='primary'
                )
        
        except Exception as e:
            print(f"⚠️ Modèle principal échoué: {e}")
            print(f"   Basculement vers le modèle de secours...")
            
            # Fallback vers le modèle de secours
            try:
                with dspy.settings.context(lm=self.fallback_lm):
                    predictor = dspy.ChainOfThought(self.signature)
                    result = predictor(ticket=ticket)
                    
                    return dspy.Prediction(
                        category=result.category,
                        priority=result.priority,
                        model_used='fallback'
                    )
            
            except Exception as fallback_error:
                # Les deux modèles ont échoué
                raise Exception(f"Échec des deux modèles. Fallback error: {fallback_error}")

print("✅ Classe FallbackTicketClassifier définie")

### Test du classifier avec fallback

In [ ]:
print("🔀 Test du classifier avec fallback")
print("   Modèle principal: llama3.1:8b")
print("   Modèle de secours: mistral:7b\n")

# Créer le classifier avec fallback
fallback_classifier = FallbackTicketClassifier(
    primary_lm=lm_ollama_llama,
    fallback_lm=lm_ollama_mistral
)

# Tester (normalement, le modèle principal devrait fonctionner)
test_ticket = "Mon imprimante ne fonctionne plus"
result = fallback_classifier(ticket=test_ticket)

print(f"Ticket: {test_ticket}")
print(f"Résultat: {result.category} | {result.priority}")
print(f"Modèle utilisé: {result.model_used}")
print()

# Note: Dans un cas réel, le fallback se déclencherait si:
# - L'API est indisponible
# - Le rate limit est atteint
# - Le timeout est dépassé
# - Une erreur réseau se produit

print("💡 Le pattern de fallback garantit la disponibilité du service")
print("   même en cas de problème avec le modèle principal.")

## 6.5 Pattern de raffinement itératif (Refine)

`dspy.Refine` est un module qui améliore itérativement une prédiction en :
1. **Exécutant le module N fois** avec différents paramètres
2. **Évaluant chaque prédiction** avec une fonction de récompense
3. **Retournant la meilleure prédiction** ou s'arrêtant si le seuil est atteint

### Différence avec les autres modules :

| Module | Approche |
|--------|----------|
| `ChainOfThought` | Une seule tentative avec raisonnement |
| `ReAct` | Raisonnement + actions séquentielles |
| `Refine` | **Plusieurs tentatives évaluées par score de qualité** |

### Cas d'usage idéaux :
- Classifications **ambiguës** nécessitant plusieurs analyses
- Situations où la **qualité est critique** et on peut se permettre plus de calculs
- Tâches avec des **critères de qualité mesurables** (via fonction de récompense)


In [ ]:
class RefinedTicketClassifier(dspy.Module):
    """
    Classifier avec raffinement itératif.
    
    Utilise dspy.Refine pour exécuter le classifier plusieurs fois
    et retourner la meilleure prédiction selon une fonction de récompense.
    """
    
    def __init__(self, N=3, threshold=1.0):
        super().__init__()
        self.valid_categories = {'hardware', 'software', 'network', 'account', 'application'}
        self.valid_priorities = {'critical', 'urgent', 'high', 'medium', 'low'}
        
        # Module de base à raffiner
        base_module = dspy.ChainOfThought(TicketClassifier)
        
        # Créer le module Refine avec fonction de récompense
        self.refine = dspy.Refine(
            module=base_module,
            N=N,  # Nombre de tentatives
            reward_fn=self._reward_function,  # Fonction d'évaluation
            threshold=threshold  # Arrêt anticipé si atteint
        )
    
    def _reward_function(self, args, prediction):
        """
        Fonction de récompense : évalue la qualité de la prédiction.
        
        Retourne un score entre 0.0 et 1.0 :
        - 1.0 : catégorie ET priorité valides
        - 0.5 : seulement l'un des deux valide
        - 0.0 : les deux invalides
        """
        score = 0.0
        
        # Vérifier la validité de la catégorie
        if hasattr(prediction, 'category'):
            if prediction.category.strip().lower() in self.valid_categories:
                score += 0.5
        
        # Vérifier la validité de la priorité
        if hasattr(prediction, 'priority'):
            if prediction.priority.strip().lower() in self.valid_priorities:
                score += 0.5
        
        return score
    
    def forward(self, ticket):
        # Refine exécutera le module N fois et retournera la meilleure prédiction
        result = self.refine(ticket=ticket)
        return dspy.Prediction(
            category=result.category,
            priority=result.priority
        )

### Test du classifier avec raffinement

In [ ]:
print("🔄 Test du classifier avec raffinement itératif\n")

# Créer le classifier avec 3 tentatives
refined_classifier = RefinedTicketClassifier(N=3, threshold=1.0)

# Ticket ambigu pour tester le raffinement
ticket_ambigu = "Le système affiche des messages d'erreur étranges. Parfois ça marche, parfois non."

print(f"Ticket: {ticket_ambigu}\n")

# Classifier avec raffinement
result = refined_classifier(ticket=ticket_ambigu)

print(f"✅ Résultat après raffinement:")
print(f"   Catégorie: {result.category}")
print(f"   Priorité: {result.priority}")

print("\n💡 Le module Refine a exécuté le classifier jusqu'à 3 fois")
print("   et a retourné la prédiction avec le meilleur score de qualité.")

### 📊 Comparaison : ChainOfThought vs Refine

| Aspect | ChainOfThought | Refine |
|--------|----------------|--------|
| **Nombre de tentatives** | 1 | N (configurable) |
| **Évaluation** | Aucune | Fonction de récompense |
| **Coût (appels LLM)** | Faible (1x) | Plus élevé (Nx) |
| **Qualité** | Bonne | Potentiellement meilleure |
| **Cas d'usage** | Tâches standard | Tâches critiques/ambiguës |

### 🎯 Quand utiliser Refine ?

✅ **Utilisez Refine quand :**
- La qualité est **critique** (décisions importantes)
- Les tickets sont **ambigus** ou complexes
- Vous avez une **fonction de qualité** claire
- Le **coût supplémentaire** est acceptable

❌ **Évitez Refine si :**
- Vous avez des **contraintes de latence** strictes
- Les **coûts** doivent être minimisés
- La tâche est **simple** et ChainOfThought suffit


## 6.6 Pattern d'ensemble (combiner plusieurs prédictions)

Le **pattern d'ensemble** combine les prédictions de plusieurs modèles pour améliorer la robustesse et la précision.

### Problème

- Un seul modèle peut faire des erreurs
- Différents modèles ont différentes forces
- Vous voulez maximiser la fiabilité

### Solution

Créer un module qui :
1. Exécute la prédiction avec **plusieurs modèles**
2. **Combine** les résultats (vote majoritaire, moyenne pondérée, etc.)
3. Retourne la prédiction la plus consensuelle

### Stratégies de combinaison

- **Vote majoritaire** : La prédiction la plus fréquente
- **Vote pondéré** : Chaque modèle a un poids différent
- **Consensus strict** : Tous les modèles doivent être d'accord

### Exemple: classifier d'ensemble

In [ ]:
from collections import Counter

class EnsembleTicketClassifier(dspy.Module):
    """
    Classifier d'ensemble utilisant plusieurs modèles et vote majoritaire
    """
    
    def __init__(self, models):
        """
        Args:
            models: Liste de tuples (lm, weight) où weight est le poids du modèle
        """
        super().__init__()
        self.models = models
        self.signature = TicketClassifier
    
    def forward(self, ticket):
        predictions = []
        
        # Obtenir les prédictions de chaque modèle
        for lm, weight in self.models:
            try:
                with dspy.settings.context(lm=lm):
                    predictor = dspy.ChainOfThought(self.signature)
                    result = predictor(ticket=ticket)
                    
                    # Ajouter la prédiction avec son poids
                    for _ in range(weight):
                        predictions.append({
                            'category': result.category.strip().lower(),
                            'priority': result.priority.strip().lower()
                        })
            
            except Exception as e:
                print(f"⚠️ Erreur avec un modèle: {e}")
                continue
        
        if not predictions:
            raise Exception("Aucun modèle n'a pu faire de prédiction")
        
        # Vote majoritaire pour la catégorie
        categories = [p['category'] for p in predictions]
        category_counts = Counter(categories)
        winning_category = category_counts.most_common(1)[0][0]
        
        # Vote majoritaire pour la priorité
        priorities = [p['priority'] for p in predictions]
        priority_counts = Counter(priorities)
        winning_priority = priority_counts.most_common(1)[0][0]
        
        # Normaliser les résultats
        category = next((c for c in CATEGORIES if c.lower() == winning_category), winning_category)
        priority = next((p for p in PRIORITIES if p.lower() == winning_priority), winning_priority)
        
        # Calculer la confiance (pourcentage d'accord)
        category_confidence = category_counts[winning_category] / len(predictions)
        priority_confidence = priority_counts[winning_priority] / len(predictions)
        
        return dspy.Prediction(
            category=category,
            priority=priority,
            category_confidence=category_confidence,
            priority_confidence=priority_confidence,
            num_models=len(self.models)
        )

print("✅ Classe EnsembleTicketClassifier définie")

### Test du classifier d'ensemble

In [ ]:
print("🎯 Test du classifier d'ensemble")
print("   Combinaison de 3 modèles avec vote majoritaire\n")

# Créer l'ensemble avec 3 modèles
# Format: (modèle, poids)
ensemble_classifier = EnsembleTicketClassifier([
    (lm_ollama_llama, 2),    # Poids 2 (meilleur modèle)
    (lm_ollama_qwen, 2),     # Poids 2 (aussi très bon)
    (lm_ollama_mistral, 1)   # Poids 1 (plus rapide mais moins précis)
])

# Tester sur quelques exemples
test_tickets = [
    "Mon ordinateur portable ne démarre plus, présentation urgente dans 1h",
    "Besoin d'accès au VPN pour le télétravail",
]

for i, ticket in enumerate(test_tickets, 1):
    print(f"{i}. Ticket: {ticket}")
    result = ensemble_classifier(ticket=ticket)
    print(f"   → Catégorie: {result.category} (confiance: {result.category_confidence:.1%})")
    print(f"   → Priorité: {result.priority} (confiance: {result.priority_confidence:.1%})")
    print()

# Évaluer sur l'ensemble de validation
print("📊 Évaluation de l'ensemble sur la validation:")
score_ensemble = evaluate_module(ensemble_classifier, val_examples, exact_match_metric)
print(f"   Score: {score_ensemble:.2%}")
print()

print("💡 L'ensemble combine les forces de plusieurs modèles")
print("   pour améliorer la robustesse et la précision.")

## 6.7 Combiner les patterns

En production, vous pouvez **combiner plusieurs patterns** pour maximiser la robustesse :

### Exemple: classifier de production complet

```python
class ProductionTicketClassifier(dspy.Module):
    """
    Classifier de production avec tous les patterns
    """
    
    def __init__(self, primary_lm, fallback_lm, ensemble_models):
        super().__init__()
        self.primary_lm = primary_lm
        self.fallback_lm = fallback_lm
        self.ensemble_models = ensemble_models
        self.valid_categories = set(cat.lower() for cat in CATEGORIES)
        self.valid_priorities = set(pri.lower() for pri in PRIORITIES)
    
    def forward(self, ticket, use_ensemble=False):
        # 1. Décider de la stratégie
        if use_ensemble:
            # Utiliser l'ensemble pour les cas critiques
            classifier = EnsembleTicketClassifier(self.ensemble_models)
        else:
            # Utiliser le fallback pour les cas normaux
            classifier = FallbackTicketClassifier(self.primary_lm, self.fallback_lm)
        
        # 2. Exécuter avec retry
        max_retries = 3
        for attempt in range(max_retries):
            try:
                result = classifier(ticket=ticket)
                
                # 3. Valider et corriger
                category, priority, is_valid = self.validate_and_correct(
                    result.category,
                    result.priority
                )
                
                return dspy.Prediction(
                    category=category,
                    priority=priority,
                    is_valid=is_valid,
                    strategy='ensemble' if use_ensemble else 'fallback',
                    attempts=attempt + 1
                )
                
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                time.sleep(2 ** attempt)
```

### Quand utiliser chaque pattern

| Pattern | Quand l'utiliser | Coût |
|---------|-----------------|------|
| **Validation** | Toujours (critique) | Faible |
| **Retry** | APIs externes | Faible |
| **Fallback** | Haute disponibilité requise | Moyen |
| **Ensemble** | Précision maximale requise | Élevé |
| **Combinaison** | Production critique | Très élevé |

## 6.8 Résumé de la Partie 6

### Ce que nous avons appris

1. **Pattern de validation** :
   - Vérifier que les sorties respectent les contraintes
   - Corriger automatiquement les valeurs invalides
   - Normaliser les formats

2. **Pattern de retry** :
   - Réessayer automatiquement en cas d'erreur temporaire
   - Backoff exponentiel (1s, 2s, 4s, 8s...)
   - Nombre maximum de tentatives

3. **Pattern de fallback** :
   - Modèle de secours si le principal échoue
   - Garantir la disponibilité du service
   - Compromis qualité/disponibilité

4. **Pattern d'ensemble** :
   - Combiner plusieurs modèles (vote majoritaire)
   - Améliorer la robustesse
   - Mesurer la confiance

5. **Combinaison de patterns** :
   - Utiliser plusieurs patterns ensemble
   - Architecture de production robuste
   - Adapter selon les besoins

### Points clés à retenir

- ✅ **Validation** : Toujours valider les sorties en production
- ✅ **Retry** : Gérer les erreurs temporaires automatiquement
- ✅ **Fallback** : Avoir un plan B pour la disponibilité
- ✅ **Ensemble** : Utiliser pour les cas critiques nécessitant haute précision
- ✅ **Combinaison** : Adapter les patterns selon vos contraintes

### Compromis à considérer

| Aspect | Simple | Production robuste |
|--------|--------|-------------------|
| **Complexité du code** | Faible | Élevée |
| **Coût d'exécution** | Faible | Élevé (multiple LLM calls) |
| **Latence** | Rapide | Plus lente |
| **Fiabilité** | Moyenne | Très élevée |
| **Maintenance** | Simple | Plus complexe |

### Recommandations

**Pour débuter** :
- Commencer simple (sans patterns)
- Ajouter la validation dès que possible
- Ajouter les autres patterns selon les besoins

**Pour la production** :
- Validation : **obligatoire**
- Retry : **fortement recommandé** pour les APIs
- Fallback : selon l'importance de la disponibilité
- Ensemble : uniquement si la précision est critique

**Stratégie progressive** :
1. Phase 1 : Module simple + validation
2. Phase 2 : Ajouter retry pour gérer les erreurs
3. Phase 3 : Ajouter fallback pour la disponibilité
4. Phase 4 : Ensemble pour les cas critiques seulement

### Prochaines étapes

- **Partie 7** : GEPA en pratique (optimisation sophistiquée avec algorithmes génétiques)
- **Partie 8** : Conclusion et mise en production